# Hard spheres in 2D

In [1]:
import numpy as np
import plotly as py
from plotly.graph_objs import *

"""basic definitions"""
DISK_RADIUS = 1


def collision(x, config, x_pos=-1):
    """return True if the disk in position x collide with any other in config
        optional parameter:
        - x_pos: the position of x in config (if present)"""

    for i in range(len(config)):
        if i != x_pos and np.sqrt(np.sum((x-config[i])**2)) < 2*DISK_RADIUS:
            return True

    return False


def random_position(size):
    """random position in a square (uniform sampling)"""
    return np.random.uniform(0, size, 2)


def configuration_direct_sampling(N, system_size):
    """direct uniform extraction of N non-overlapping disks in a square"""
    config = []


    for i in range(N):
        x = random_position(system_size)
        y = [0, system_size, -system_size]
        x_pbc_clones = [ x + k for k in np.transpose([np.tile(y, len(y)), np.repeat(y, len(y))])]

        for j in x_pbc_clones:
            if collision(np.array(j) , np.array(config)):
                rejected_counter[0] += 1
                return configuration_direct_sampling(N, system_size)

        config += [x]

    return np.array(config)



def generate_square_lattice(system_size, num_disks):
    """Generates a square lattice with a given number of disks"""
    ss_disks = np.sqrt(num_disks)
    step = system_size / ss_disks
    x = [ step*(.5 + i) for i in range(int(ss_disks))]
    return np.transpose([np.tile(x, len(x)), np.repeat(x, len(x))])


def plot_config(x, y, system_size):
    """plot the configuration of the system"""
    trace = Scatter(
        x=x,
        y=y,
        mode = 'markers',
        marker = dict(size = 34.5*40.1060523941/system_size, sizemode = "diameter")
        )

    lyt = Layout(
        yaxis = dict(scaleanchor = "x"),
        shapes=[{'type': 'square', 'x0': 0, 'y0': 0, 'x1': system_size, 'y1': system_size, 'xref':'x', 'yref':'y'}]
        )

    fig = dict(data=[trace], layout=lyt)

    py.offline.plot(fig)



def system_size(num_disks, density, disk_area):
    """compute the system size to have that num_disks and density"""
    return np.sqrt(num_disks * disk_area / density)

def density(num_disks, syst_size):
    """compute the density of disks (the fraction of area occupied by disks)"""
    return np.pi * num_disks / syst_size**2


def clone_collision(x, config, system_size):
    """check if the disk is in the area in which is periodic clones can
       undergo collisions"""

    position = (x < 2*DISK_RADIUS) -1*(x > system_size - 2*DISK_RADIUS)

    if np.sum(np.abs(position)):
        clone = x + system_size*position
        if collision(clone, config):
            return True

    if np.sum(np.abs(position)) == 2:
        clone1 = x + system_size*np.array([0, position[1]])
        clone2 = x + system_size*np.array([position[0], 0])
        if collision(clone1, config) or collision(clone2, config):
            return True

    return False


def reposition_disk(x, system_size):
    """reposition the disk inside the square if it's out"""
    out = np.array(-1*(x > system_size) + (x < 0))

    if np.sum(np.abs(out)):
        x += system_size * out

    return x


def move_disk(config, num_disks, step_size, system_size):
    """try to move a random disk of the current config"""
    # move proposal
    disk = np.random.randint(num_disks)
    step = np.random.uniform(-step_size, step_size, 2)
    final_position = config[disk] + step

    # check if there are collision (considering also the PBCs)
    if collision(final_position, config, disk) \
        or clone_collision(final_position, config, system_size):
        return config

    # reposition the disk if it is out
    final_position = reposition_disk(final_position, system_size)

    config[disk] = final_position

    return config


We first try to implement a direct sampling strategy. We expect that reaching large density will be practically impossible, since the probability of obtaining two overlapping disk in the extracted configuration will be very high (the acceptance will tend to zero).

In [2]:
rejected_counter = [0]
num_disks = 43
syst_size = 40
c = configuration_direct_sampling(num_disks, syst_size)

plot_config(c[:,0], c[:,1], syst_size)
print("For a density of disks of {0:4f} we get an acceptance ratio of 1/{1}". \
      format(density(num_disks, syst_size), rejected_counter[0] + 1))

For a density of disks of 0.084430 we get an acceptance ratio of 1/358


As we can see even for small density the computational cost of obtaining a single configuration is very high. Also the presence of the periodic boundary condictions (PBCs) increases the computational cost since, for each step, we have to check if the new disk extracted collides with any other (due to the PBCs if a disk go beyond the square border can collide with the disks near the other side of the square). In this first approach we have controlled for each step if any of the particles duplicate (generated by the PBCs) collided with any other particle. In the following we'll do better, doing this check only when the disk is sufficiently near to the border (less than 2 times its radius).

## Markov chain Montecarlo with square lattice initial condiction


In [4]:
"""system properties"""
for density in [0.5, 0.72]:
    
    num_disks = 16**2
    disk_area = np.pi * DISK_RADIUS**2

    syst_size = system_size(num_disks, density, disk_area)
    
    config = generate_square_lattice(syst_size, num_disks)

    NUM_STEPS = 10000000

    for i in range(NUM_STEPS + 1):
        config = move_disk(config, num_disks, 0.5*DISK_RADIUS, syst_size)
        if i%100000 == 0:
            plot_config(config[:,0], config[:,1], syst_size)


#config after NUM_STEPS steps
# plot_config(config[:,0], config[:,1], syst_size)


Ater some trial we get an acceptance of about 0.5 for:
- a step size of 0.65*DISK_RADIUS for density=0.5
- a step size 0.11*DISK_RADIUS for density=0.72
for the a density of 0.5 nothing particular happens: the system get disordered and then remains desordered, behaving like a liquid in some way
when the density increases we can see that after long enough domains of ordered (aligned) particles arise in the system (that behaves like a solid).